In [24]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import urllib
import os
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import altair as alt

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

In [4]:
%%time
cars = pd.read_csv('road_eqs_carmot/road_eqs_carmot_1_Data.csv', encoding= 'unicode_escape') 
print("Finished reading: {}".format(len(cars)))

Finished reading: 12180
CPU times: user 15 ms, sys: 3.67 ms, total: 18.7 ms
Wall time: 17.3 ms


In [7]:
cars.head(20)

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
0,1990,Belgium,Number,Total,Total,3 864 000
1,1990,Belgium,Number,Total,Less than 1 400 cm³,:
2,1990,Belgium,Number,Total,From 1 400 to 1 999 cm³,:
3,1990,Belgium,Number,Total,2 000 cm³ or over,:
4,1990,Belgium,Number,Petroleum products,Total,:
5,1990,Belgium,Number,Petroleum products,Less than 1 400 cm³,:
6,1990,Belgium,Number,Petroleum products,From 1 400 to 1 999 cm³,:
7,1990,Belgium,Number,Petroleum products,2 000 cm³ or over,:
8,1990,Belgium,Number,Diesel,Total,:
9,1990,Belgium,Number,Diesel,Less than 1 400 cm³,:


In [225]:
cars_eng_total = cars[cars['ENGINE'] == "Total"] 

In [226]:
cars_eng_total.head()

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
0,1990,Belgium,Number,Total,Total,3 864 000
4,1990,Belgium,Number,Petroleum products,Total,:
8,1990,Belgium,Number,Diesel,Total,:
12,1990,Bulgaria,Number,Total,Total,1 317 000
16,1990,Bulgaria,Number,Petroleum products,Total,:


In [240]:
cars_total = cars_eng_total[cars_eng_total['MOT_NRG'] == "Total"] 

In [241]:
cars_total["TIME"] = pd.to_datetime(cars_total["TIME"], format='%Y')
#cars_total["TIME"] = cars_total["TIME"].year # pd.DatetimeIndex(cars_total["TIME"]).year
cars_total["TIME"].dtype


dtype('<M8[ns]')

In [242]:
cars_total["TIME"].head()

0    1990-01-01
12   1990-01-01
24   1990-01-01
36   1990-01-01
48   1990-01-01
Name: TIME, dtype: datetime64[ns]

In [243]:
cars_total["Value"] = cars_total["Value"].str.replace(" ","")

In [244]:
cars_total["Value"] = cars_total["Value"].replace(":",np.nan)

In [310]:
#cars_total["Value"] = cars_total["Value"].astype(int)
cars_total.head(34)

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
0,1990-01-01,Belgium,Number,Total,Total,3864000
12,1990-01-01,Bulgaria,Number,Total,Total,1317000
24,1990-01-01,Czechia,Number,Total,Total,2411000
36,1990-01-01,Denmark,Number,Total,Total,1590000
48,1990-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,30685000
60,1990-01-01,Estonia,Number,Total,Total,241664
72,1990-01-01,Ireland,Number,Total,Total,797713
84,1990-01-01,Greece,Number,Total,Total,1736000
96,1990-01-01,Spain,Number,Total,Total,11996000
108,1990-01-01,France,Number,Total,Total,23550000


In [246]:
cars_DE = cars_total[cars_total['GEO'] == "Germany (until 1990 former territory of the FRG)"] 

In [278]:
cars_DE = cars_DE[cars_DE['Value'].notna()]
cars_DE["Value"] = cars_DE["Value"].astype(int)
cars_DE.head(5)

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
48,1990-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,30685000
468,1991-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,31322000
1308,1993-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,38772000
1728,1994-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,39765000
2148,1995-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,40404000


In [279]:
cars_DE.tail()

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
10128,2014-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,44403000
10548,2015-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,45071000
10968,2016-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,45803560
11388,2017-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,46474594
11808,2018-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,47095784


In [331]:
alt.Chart(cars_DE).mark_line(point=True).properties(width=700, height=500).encode(
    alt.X('TIME', scale=alt.Scale(domain=('1990','2018')), title='Year'),
    alt.Y('Normalized', scale=alt.Scale(domain=(100,200)), title='Passenger Cars'),
    alt.Color('GEO', title='Country')
) 

alt.Chart(...)

In [262]:
cars_NO = cars_total[cars_total['GEO'] == "Norway"] 
cars_NO = cars_NO[cars_NO['Value'].notna()]
cars_NO["Value"] = cars_NO["Value"].astype(int)
cars_NO.head(30)

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
360,1990-01-01,Norway,Number,Total,Total,1613000
780,1991-01-01,Norway,Number,Total,Total,1615000
1200,1992-01-01,Norway,Number,Total,Total,1619000
1620,1993-01-01,Norway,Number,Total,Total,1633000
2040,1994-01-01,Norway,Number,Total,Total,1654000
2460,1995-01-01,Norway,Number,Total,Total,1685000
2880,1996-01-01,Norway,Number,Total,Total,1661000
3300,1997-01-01,Norway,Number,Total,Total,1758000
3720,1998-01-01,Norway,Number,Total,Total,1786000
4140,1999-01-01,Norway,Number,Total,Total,1814000


In [263]:
cars_NO.dtypes

TIME       datetime64[ns]
GEO                object
UNIT               object
MOT_NRG            object
ENGINE             object
Value               int64
dtype: object

In [327]:
alt.Chart(cars_NO).mark_line(point=True).properties(width=700, height=500).encode(
    alt.X('TIME', scale=alt.Scale(domain=('1990','2018')), title='Year'),
    alt.Y('Normalized', scale=alt.Scale(domain=(100,200)), title='Passenger Cars'),
    alt.Color('GEO', title='Country')
) 

alt.Chart(...)

In [283]:
#cars_DE["%"] = cars_DE.apply(cars_DE.Value / 30685000)  

In [284]:
cars_CH = cars_total[cars_total['GEO'] == "Switzerland"] 
cars_CH = cars_CH[cars_CH['Value'].notna()]
cars_CH["Value"] = cars_CH["Value"].astype(int)
cars_CH.head()

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
372,1990-01-01,Switzerland,Number,Total,Total,2985000
792,1991-01-01,Switzerland,Number,Total,Total,3058000
1212,1992-01-01,Switzerland,Number,Total,Total,3091000
1632,1993-01-01,Switzerland,Number,Total,Total,3110000
2052,1994-01-01,Switzerland,Number,Total,Total,3165000


In [329]:
alt.Chart(cars_CH).mark_line(point=True).properties(width=700, height=500).encode(
    alt.X('TIME', scale=alt.Scale(domain=('1990','2018')), title='Year'),
    alt.Y('Normalized', scale=alt.Scale(domain=(100,200)), title='Passenger Cars'),
    alt.Color('GEO', title='Country')
) 

alt.Chart(...)

In [287]:
cars_FR = cars_total[cars_total['GEO'] == "France"] 
cars_FR = cars_FR[cars_FR['Value'].notna()]
cars_FR["Value"] = cars_FR["Value"].astype(int)
cars_FR.head()

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
108,1990-01-01,France,Number,Total,Total,23550000
528,1991-01-01,France,Number,Total,Total,23810000
948,1992-01-01,France,Number,Total,Total,24020000
1368,1993-01-01,France,Number,Total,Total,24385000
1788,1994-01-01,France,Number,Total,Total,24900000


In [320]:
cars_FR.tail()
cars_FR.head()

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value,Normalized
108,1990-01-01,France,Number,Total,Total,23550000,100.000000
528,1991-01-01,France,Number,Total,Total,23810000,101.104034
948,1992-01-01,France,Number,Total,Total,24020000,101.995754
1368,1993-01-01,France,Number,Total,Total,24385000,103.545648
1788,1994-01-01,France,Number,Total,Total,24900000,105.732484


In [325]:
alt.Chart(cars_FR).mark_line(point=True).properties(width=700, height=500).encode(
    alt.X('TIME', scale=alt.Scale(domain=('1990','2018')), title='Year'),
    alt.Y('Normalized', scale=alt.Scale(domain=(100,200)), title='Passenger Cars'),
    alt.Color('GEO', title='Country')
) 

alt.Chart(...)

In [300]:
cars_total = cars_total[cars_total['Value'].notna()]
cars_total["Value"] = cars_total["Value"].astype(int)
cars_total

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
0,1990-01-01,Belgium,Number,Total,Total,3864000
12,1990-01-01,Bulgaria,Number,Total,Total,1317000
24,1990-01-01,Czechia,Number,Total,Total,2411000
36,1990-01-01,Denmark,Number,Total,Total,1590000
48,1990-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,30685000
...,...,...,...,...,...,...
12120,2018-01-01,Norway,Number,Total,Total,2751949
12132,2018-01-01,Switzerland,Number,Total,Total,4602700
12144,2018-01-01,North Macedonia,Number,Total,Total,415062
12156,2018-01-01,Turkey,Number,Total,Total,12398190


In [330]:
#cars_FR["Normalized"] = cars_FR["Value"]/(0.01*cars_FR["Value"].iloc[0])
#cars_NO["Normalized"] = cars_NO["Value"]/(0.01*cars_NO["Value"].iloc[0])
#cars_CH["Normalized"] = cars_CH["Value"]/(0.01*cars_CH["Value"].iloc[0])
cars_DE["Normalized"] = cars_DE["Value"]/(0.01*cars_DE["Value"].iloc[0])

In [301]:
cars_total.groupby(by="GEO").head()

,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
0,1990-01-01,Belgium,Number,Total,Total,3864000
12,1990-01-01,Bulgaria,Number,Total,Total,1317000
24,1990-01-01,Czechia,Number,Total,Total,2411000
36,1990-01-01,Denmark,Number,Total,Total,1590000
48,1990-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,30685000
...,...,...,...,...,...,...
2124,1995-01-01,Czechia,Number,Total,Total,3043320
2148,1995-01-01,Germany (until 1990 former territory of the FRG),Number,Total,Total,40404000
2544,1996-01-01,Czechia,Number,Total,Total,3192532
11748,2017-01-01,Kosovo (under United Nations Security Council ...,Number,Total,Total,168759


,TIME,GEO,UNIT,MOT_NRG,ENGINE,Value
108,1990-01-01,France,Number,Total,Total,23550000
